In [2]:
!pip install pymysql boto3
import boto3
from configparser import ConfigParser
import pandas as pd
import pymysql
import io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 9.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.2 MB/s eta 0:00:00


In [3]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [13]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_context = config.get("Bucket", "bucket_context")

# Configurações para conexão com MySQL
# mysql_host = config.get('MYSQL', 'endpoint')
mysql_host = 'mysql'
mysql_user = config.get('MYSQL', 'user')
mysql_pwd = config.get('MYSQL', 'user_pwd')
mysql_db_name = config.get('MYSQL', 'db_name')

# Nome do arquivo a ser lido
source_filename = config.get("FILE", "coordenadas_municipios_csv")

In [14]:
print("endpoint:", endpoint)
print("access_key:", access_key)
print("secret_key:", secret_key)
print("bucket_context:", bucket_context)
print("mysql_host:", mysql_host)
print("mysql_user:", mysql_user)
print("mysql_pwd:", mysql_pwd)
print("mysql_db_name:", mysql_db_name)
print("source_filename:", source_filename)

endpoint: http://minio:9000
access_key: aulafia
secret_key: aulafia@123
bucket_context: context
mysql_host: mysql
mysql_user: root
mysql_pwd: root
mysql_db_name: db_aulafia
source_filename: Coordenadas.csv


In [9]:
# Inicializar o cliente boto3 para S3
minio_client = boto3.client("s3", 
                            endpoint_url=endpoint,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key
)

In [10]:
# Baixar o arquivo CSV do bucket 'context'
response = minio_client.get_object(Bucket=bucket_context,
                         Key=source_filename)
csv_content = response['Body'].read()

In [11]:
# Carregar o conteúdo CSV em um DataFrame do pandas
data_frame = pd.read_csv(io.BytesIO(csv_content))

In [18]:
# Conectar ao MySQL
mysql_connection = pymysql.connect(host=mysql_host,
                                   user=mysql_user,
                                   password=mysql_pwd,
                                   db=mysql_db_name,
                                   cursorclass=pymysql.cursors.DictCursor)

In [19]:
# Fazer a ingestão na tabela
try:
    # Apagar a tabela coordenadas (se existir)
    with mysql_connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS coordenadas")

    # Criar novamente a tabela coordenadas
    with mysql_connection.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE coordenadas (
                codigo_ibge INT NOT NULL,
                nome VARCHAR(100) NOT NULL,
                latitude FLOAT(8) NOT NULL,
                longitude FLOAT(8) NOT NULL,
                capital BOOLEAN NOT NULL,
                codigo_uf INT NOT NULL,
                siafi_id VARCHAR(4) NOT NULL UNIQUE,
                ddd INT NOT NULL,
                fuso_horario VARCHAR(32) NOT NULL,
                PRIMARY KEY (codigo_ibge)
            )
        """)
    
    # Inserir os dados na tabela coordenadas
    with mysql_connection.cursor() as cursor:
        for index, row in data_frame.iterrows():
            sql = "INSERT INTO coordenadas VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            values = (row['codigo_ibge'], row['nome'], row['latitude'], row['longitude'],
                      row['capital'], row['codigo_uf'], row['siafi_id'], row['ddd'], row['fuso_horario'])
            cursor.execute(sql, values)
        mysql_connection.commit()

    print(f'Dados do arquivo {source_filename} inseridos na tabela coordenadas com sucesso.')

finally:
    mysql_connection.close()

Dados do arquivo Coordenadas.csv inseridos na tabela coordenadas com sucesso.
